In [2]:
print "Importing packages"

import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn as learn
import xgboost
import pandas as pd
import numpy as np
import re


Importing packages


Third attempt at the DonorsChoose Dataset. Very basic stacking model.
Anisotropic's notebook on Stacking\ensembling was really useful for creating this.

In [3]:
#reading data

train= pd.read_csv("/home/souravg/train.csv")

test=pd.read_csv("/home/souravg/test.csv") 

resource= pd.read_csv("/home/souravg/resources.csv")

#print train.head(7)

#print resource.head(7)


joined= pd.merge(train,resource,how='inner',on='id',sort=False,)

joined_test=pd.merge(test,resource,how='inner',on='id',sort=False,)


#print joined.columns

#print joined.head(5)

#train.school_state.astype("category")

#train.project_grade_category.astype("category")

#train.project_subject_categories.astype("category")

#train.project_subject_subcategories.astype("category")

print'--------------------------------------------------'




/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


--------------------------------------------------


In [4]:
joined['Total_sum']= joined.quantity*joined.price
joined_test['Total_sum']= joined_test.quantity*joined_test.price
#print joined.head(3)

print '-------------------------------'

#print joined_test.head(3)


-------------------------------


In [5]:
sumdf= joined.groupby('id',as_index=False).agg({'Total_sum':'sum'})

sumdf_test=joined_test.groupby('id',as_index=False).agg({'Total_sum':'sum'})

#print sumdf.head(3)
print '-----------------------'
#print sumdf_test.head(3)

#print joined.Total_sum.head(3)

-----------------------


In [6]:
final_train= pd.merge(train,sumdf,on='id',how='inner',sort=False,)
final_test=pd.merge(test,sumdf_test,on='id',how='inner',sort=False,)

#print final_train.head(3)
print '-------------------'
#print final_test.head(3)

-------------------


The cell above read the train data, converted appropriate parameters to categorical type & did some group by counts. From the results it seems like California, New York, Texas & Florida have the largest number of projects.
Grades PreK-2 had the most number of projects, & the number of projects decrease monotonically with increasing age of students. Grades 9-12 have the fewest number of projects. I must say that this came as a surprise, I would hae thought seniors would need more funding.
There seems to be the following project categories:-

Applied Learning
Health & Sports
History & Civics
Math & Science
Music & The Arts
Special Needs
Warmth, Care & Hunger

But the majority of projects are tagged with multiple categories, hence there are lot more different project categories in the data (combinations of the base categories).

Math & Science, Literacy & Language & Health & Sport seem to be leaders in terms of project count.

In [7]:
train=final_train
test=final_test

ftest=test.drop([u'id', u'teacher_id', u'teacher_prefix',u'project_title', u'project_essay_1', u'project_essay_2',
       u'project_essay_3', u'project_essay_4', u'project_resource_summary','project_submitted_datetime','project_subject_subcategories'],1)

ftrain=train.drop([u'id', u'teacher_id', u'teacher_prefix',u'project_title', u'project_essay_1', u'project_essay_2',
       u'project_essay_3', u'project_essay_4', u'project_resource_summary','project_submitted_datetime','project_subject_subcategories'],1)

target=ftrain.project_is_approved

ftrain=ftrain.drop(['project_is_approved'],1)

print '----------------------------------'

copy=pd.DataFrame(pd.concat([ftrain,ftest]))

copy["school_state"]=copy["school_state"].astype('category')
copy["project_grade_category"]=copy["project_grade_category"].astype('category')
copy["project_subject_categories"]=copy["project_subject_categories"].astype('category')
print '&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&'
cat_columns = copy.select_dtypes(['category']).columns

print cat_columns

copy[cat_columns] = copy[cat_columns].apply(lambda x: x.cat.codes)

#copy['school_state']=copy['school_state'].cat.codes
#copy['project_grade_category']=copy['project_grade_category'].cat.codes
#copy['project_subject_categories']=copy['project_subject_categories'].cat.codes
#copy['project_subject_subcategories']=copy['project_subject_subcategories'].cat.codes
print '&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&'
print 'copy',copy["school_state"].describe()
print 'copy',copy["project_grade_category"].describe()
print 'copy',copy["project_subject_categories"].describe()
print '[[[[[[[[[[[[[[[[[]]]]]]]]]]]]]]]]]'

#copy=copy.drop([u'school_state', u'project_grade_category', u'project_subject_categories',u'project_subject_subcategories'],1)

ftrain= copy[0:182080]
ftest= copy[182080:]

print '----------------------------'

print ftrain.head(2)
print '--------------------------------'
print ftest.head(2)


#print target.head(5)

#ftrain=pd.get_dummies(ftrain,prefix=['state','grade','categories','subcategories'])

#ftest=pd.get_dummies(ftest,prefix=['state','grade','categories','subcategories'])

print '***************************************'



----------------------------------
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Index([u'school_state', u'project_grade_category',
       u'project_subject_categories'],
      dtype='object')
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
copy count    260115.000000
mean         22.977856
std          14.924920
min           0.000000
25%           9.000000
50%          22.000000
75%          36.000000
max          50.000000
Name: school_state, dtype: float64
copy count    260115.000000
mean          1.571117
std           1.316890
min           0.000000
25%           0.000000
50%           2.000000
75%           3.000000
max           3.000000
Name: project_grade_category, dtype: float64
copy count    260115.000000
mean         25.046414
std          11.742591
min           0.000000
25%          24.000000
50%          28.000000
75%          32.000000
max          50.000000
Name: project_subject_categories, dtype: float64
[[[[[[[[[[[[[[[[[]]]]]]]]]]]]]]]]]
----------------------------
 

In [8]:

print '*****************************************'


#print '========================================== Normalization of project count & total sum'

#projectcount_mean=ftrain['teacher_number_of_previously_posted_projects'].mean()
#projectcount_sd=ftrain['teacher_number_of_previously_posted_projects'].std()

#totalsum_mean=ftrain['Total_sum'].mean()
#totalsum_sd=ftrain['Total_sum'].std()

#print projectcount_mean,' ',projectcount_sd

#ftrain['Total_sum']=(ftrain['Total_sum']-totalsum_mean)/totalsum_sd
#ftrain['teacher_number_of_previously_posted_projects']= (ftrain['teacher_number_of_previously_posted_projects']-projectcount_mean)/projectcount_sd

#ftest['Total_sum']=(ftest['Total_sum']-totalsum_mean)/totalsum_sd
#ftest['teacher_number_of_previously_posted_projects']= (ftest['teacher_number_of_previously_posted_projects']-projectcount_mean)/projectcount_sd


#print '-------------------------------- Normalization done'


from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from sklearn import ensemble
from sklearn.neural_network import MLPClassifier

abclf= ensemble.AdaBoostClassifier(n_estimators=1000,)
rfclf= ensemble.RandomForestClassifier(n_estimators=1000,criterion='entropy',max_depth=5,warm_start=True,)
gbclf= ensemble.GradientBoostingClassifier(n_estimators=1000,max_depth=4,min_samples_split=4,subsample=.3)

clf= ensemble.RandomForestClassifier(n_estimators=500,criterion='entropy',max_depth=3,warm_start=True,)
#clf = AdaBoostClassifier(n_estimators=500, learning_rate= 0.4)
#clf = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(200, 100), random_state=1,max_iter=1000)
print "Hola"



clf.fit(ftrain,target)


#abclf.fit(ftrain, target)
#rfclf.fit(ftrain, target)
#gbclf.fit(ftrain, target)

#print 'Feature importance from adaboost='
#print abclf.feature_importances_

print 'Feature importance from random forest='
print clf.feature_importances_

#print 'Feature importance from gradient boosting='
#print gbclf.feature_importances_

#tabresult=abclf.predict_proba(ftrain)
#trfresult=rfclf.predict_proba(ftrain)
#tgbresult=gbclf.predict_proba(ftrain)


#abresult=abclf.predict_proba(ftest)
#rfresult=rfclf.predict_proba(ftest)
#gbresult=gbclf.predict_proba(ftest)

print "finito"

*****************************************
Hola
Feature importance from random forest=
[ 0.01363058  0.00931885  0.15464824  0.61235736  0.21004498]
finito


In [9]:

result=clf.predict_proba(ftest)

r_df=pd.DataFrame(dict(id=test.id,project_is_approved=result[:,1]))

print r_df.shape
print r_df.head(30)
#print rfclf.feature_importances_


r_df.to_csv('/home/souravg/output5.2.csv',index=False)


(78035, 2)
         id  project_is_approved
0   p233245             0.837631
1   p096795             0.855831
2   p236235             0.829173
3   p233680             0.851656
4   p171879             0.826077
5   p016071             0.841211
6   p099906             0.863504
7   p200236             0.847926
8   p129452             0.880722
9   p186652             0.847473
10  p257992             0.855495
11  p055177             0.835415
12  p069502             0.849052
13  p099455             0.856198
14  p226195             0.869366
15  p048432             0.885633
16  p028074             0.853984
17  p188220             0.818295
18  p213507             0.851997
19  p006068             0.863422
20  p124112             0.850494
21  p190450             0.832231
22  p180156             0.820369
23  p025868             0.834366
24  p134875             0.820173
25  p169259             0.833154
26  p124445             0.850317
27  p090153             0.903948
28  p086796             0.854164

[[ 0.16478707  0.83521293]
 [ 0.13464939  0.86535061]
 [ 0.18761641  0.81238359]
 [ 0.14355027  0.85644973]
 [ 0.17657868  0.82342132]
 [ 0.15248022  0.84751978]
 [ 0.13040103  0.86959897]
 [ 0.14690793  0.85309207]
 [ 0.10957476  0.89042524]
 [ 0.14672544  0.85327456]
 [ 0.13865819  0.86134181]
 [ 0.15728328  0.84271672]
 [ 0.16674882  0.83325118]
 [ 0.13409432  0.86590568]
 [ 0.12280192  0.87719808]
 [ 0.11945113  0.88054887]
 [ 0.14228784  0.85771216]
 [ 0.19036895  0.80963105]
 [ 0.14258076  0.85741924]
 [ 0.13383068  0.86616932]
 [ 0.17531093  0.82468907]
 [ 0.16300933  0.83699067]
 [ 0.17815078  0.82184922]
 [ 0.17188554  0.82811446]
 [ 0.1812131   0.8187869 ]
 [ 0.17091193  0.82908807]
 [ 0.16154056  0.83845944]
 [ 0.08719656  0.91280344]
 [ 0.14170426  0.85829574]
 [ 0.09523673  0.90476327]]
0     p233245
1     p096795
2     p236235
3     p233680
4     p171879
5     p016071
6     p099906
7     p200236
8     p129452
9     p186652
10    p257992
11    p055177
12    p069502
13    p